In [6]:
from datetime import datetime
from pony.orm import *
import configparser
import json
import redis
set_sql_debug(True)
from PIL import Image

In [7]:
config = configparser.ConfigParser()
config.read('config.ini')

provider="mysql"

db = Database()

db.bind(
    provider=provider, 
    host=config[provider]["host"],
    user=config[provider]["user"],
    passwd=config[provider]["pass"],
    db="media_index4",
)

GET NEW CONNECTION
RELEASE CONNECTION


In [34]:
class Thumbnail(db.Entity):
    id = PrimaryKey(int, auto=True)
    xxhash = Required(str, unique=True)
    thumbnail = Optional(bytes)

db.generate_mapping()

ERDiagramError: Entity Thumbnail already exists

In [9]:
config = configparser.ConfigParser()
config.read('config.ini')
exif_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["exif"],
)
xxhash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["xxhash"],
)

In [10]:
import utils

Using example pictures from the darknet project:
https://github.com/pjreddie/darknet/wiki/YOLO:-Real-Time-Object-Detection#user-content-detection-using-a-pre-trained-model

In [11]:
from get_files import get_files

In [16]:
JPEGS=get_files(directory="/net/8600k.local/vault/pictures/", extensions=[".jpg", ".jpeg"])

In [21]:
for img_path in JPEGS:
    break
img_path

'/net/8600k.local/vault/pictures/2016/07-July/2016Jul30_092415.51.jpg'

In [27]:
import io

In [31]:
def cache_thumbnail(img_path, size = (255, 255)):
    xxhash = utils.cache_xxhash(img_path)
    img = Image.open(img_path)
    img.thumbnail(size)
    img
    with io.BytesIO() as buffer:
        img.save(buffer, format="jpeg")
        thumbnail = buffer.getvalue()
    try:
        Thumbnail(xxhash=xxhash, thumbnail=thumbnail)
        db.commit()
    except CacheIndexError:
        pass
    except:
        raise

In [32]:
for img_path in JPEGS:
    break
img_path

'/net/8600k.local/vault/pictures/2016/07-July/2016Jul30_092114.25.jpg'

In [33]:
cache_thumbnail(img_path=img_path)

Cached hash : /net/8600k.local/vault/pictures/2016/07-July/2016Jul30_092114.25.jpg
INSERT INTO `thumbnail` (`xxhash`, `thumbnail`) VALUES (%s, %s)
['6770cfa14daca66f', b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00\xaa\x00\xff\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\

COMMIT


/vault/projects/PhotoStuffs/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1300, "Invalid utf8 character string: 'FFD8FF'")
  result = self._query(query)
